# Academy Award nominations
---
In this project, we clean a CSV dataset and add it to a SQLite database.  The data can be downloaded [here](https://www.aggdata.com/awards/oscar).

Here are the columns in the dataset, academy_awards.csv:

- `Year` - the year of the awards ceremony.
- `Category` - the category of award the nominee was nominated for.
- `Nominee` - the person nominated for the award.
- `Additional Info` - this column contains additional info like:
 - the movie the nominee participated in.
 - the character the nominee played (for acting awards).
- `Won?` - this column contains either YES or NO depending on if the nominee won the award.

## Importing the data

In [1]:
import pandas as pd
data = pd.read_csv('academy_awards.csv', encoding='ISO-8859-1')

## Exploring the data

In [2]:
data.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data['Unnamed: 5'].value_counts()

*                                                                                                               7
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    1
 error-prone measurements on sets. [Digital Imaging Technology]"                                                1
 resilience                                                                                                     1
 discoverer of stars                                                                                            1
Name: Unnamed: 5, dtype: int64

In [4]:
data['Unnamed: 6'].value_counts()

*                                                                   9
 direct radiator bass style cinema loudspeaker systems. [Sound]"    1
 sympathetic                                                        1
 flexibility and water resistance                                   1
Name: Unnamed: 6, dtype: int64

In [5]:
data['Unnamed: 7'].value_counts()

 kindly                                               1
*                                                     1
 while requiring no dangerous solvents. [Systems]"    1
Name: Unnamed: 7, dtype: int64

In [6]:
data['Unnamed: 8'].value_counts()

*                                                 1
 understanding comedy genius - Mack Sennett.""    1
Name: Unnamed: 8, dtype: int64

In [7]:
data['Unnamed: 9'].value_counts()

*    1
Name: Unnamed: 9, dtype: int64

In [8]:
data['Unnamed: 10'].value_counts()

*    1
Name: Unnamed: 10, dtype: int64

The dataset is incredibly messy and one can notice many inconsistencies that make it hard to work with. Most columns don't have consistent formatting, which is incredibly important when we use SQL to query the data later on. Other columns vary in the information they convey based on the type of awards category that row corresponds to.

## Cleaning the dataset

### Year column

In [9]:
data['Year'] = data['Year'].str[0:4].astype(int)

### Nomations dataset

In [10]:
later_than_2000 = data[data['Year']>2000]

In [11]:
award_categories = ['Actor -- Leading Role', 'Actor -- Supporting Role', 'Actress -- Leading Role', 'Actress -- Supporting Role']
nominations = later_than_2000[later_than_2000['Category'].isin(award_categories)].copy()

In [12]:
nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


### Won? column

Since SQLite uses the integers 0 and 1 to represent Boolean values, we convert the Won? column to reflect this. 

In [13]:
nominations['Won?'] = nominations['Won?'].map({'YES': 1, 'NO': 0})

In [14]:
nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
nominations['Won'] = nominations['Won?']

### Dropping unnessary columns

In [16]:
final_nominations = nominations.drop(['Won?','Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], axis=1)

In [17]:
final_nominations.head()

,Year,Category,Nominee,Additional Info,Won
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0


### Splitting `Additional Info` column

The values in this column contain the movie name and the character the nominee played. Instead of keeping these values in 1 column, we split them up into 2 different columns for easier querying.

In [18]:
additional_info_one = final_nominations['Additional Info'].str.rstrip("'}")
additional_info_two = additional_info_one.str.split(" {'")

In [19]:
additional_info_two.head()

0                        [Biutiful, Uxbal]
1             [True Grit, Rooster Cogburn]
2    [The Social Network, Mark Zuckerberg]
3      [The King's Speech, King George VI]
4                [127 Hours, Aron Ralston]
Name: Additional Info, dtype: object

In [20]:
final_nominations['Movie'] = additional_info_two.str[0]
final_nominations['Character'] = additional_info_two.str[1]

In [21]:
final_nominations.head()

,Year,Category,Nominee,Additional Info,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0,127 Hours,Aron Ralston


### Dropping `Additional Info` column

In [22]:
final_nominations = final_nominations.drop(['Additional Info'], axis=1)

### Checking the result

In [23]:
final_nominations.head()

,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,0,127 Hours,Aron Ralston


## Importing the dataset into a SQL database

### Creating a Connection instance using sqlite3.connect()

In [24]:
import sqlite3
conn = sqlite3.connect('nominations.db')

### Creating the table "nominations"

In [ ]:
final_nominations.to_sql('nominations', conn, index=False)

### Querying the table

In [26]:
conn.execute('pragma table_info(nominations)').fetchall()

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]

In [27]:
conn.execute('select * from nominations limit 10').fetchall()

[(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal'),
 (2010,
  'Actor -- Leading Role',
  'Jeff Bridges',
  0,
  'True Grit',
  'Rooster Cogburn'),
 (2010,
  'Actor -- Leading Role',
  'Jesse Eisenberg',
  0,
  'The Social Network',
  'Mark Zuckerberg'),
 (2010,
  'Actor -- Leading Role',
  'Colin Firth',
  1,
  "The King's Speech",
  'King George VI'),
 (2010,
  'Actor -- Leading Role',
  'James Franco',
  0,
  '127 Hours',
  'Aron Ralston'),
 (2010,
  'Actor -- Supporting Role',
  'Christian Bale',
  1,
  'The Fighter',
  'Dicky Eklund'),
 (2010,
  'Actor -- Supporting Role',
  'John Hawkes',
  0,
  "Winter's Bone",
  'Teardrop'),
 (2010,
  'Actor -- Supporting Role',
  'Jeremy Renner',
  0,
  'The Town',
  'James Coughlin'),
 (2010,
  'Actor -- Supporting Role',
  'Mark Ruffalo',
  0,
  'The Kids Are All Right',
  'Paul'),
 (2010,
  'Actor -- Supporting Role',
  'Geoffrey Rush',
  0,
  "The King's Speech",
  'Lionel Logue')]

### Cosing the connection

In [28]:
conn.close()